In [10]:
import sbol2
from buildcompiler.abstract_translator import Plasmid
from typing import Union, List

## Helper Function:

In [11]:
def get_or_pull(doc, sbh, uri):
    """
    Get an SBOL object from a Document.
    If missing, pull it from SynBioHub and retry.
    """
    if uri not in doc:
        sbh.pull(uri, doc)
    return doc.get(uri)

In [12]:
class BuildCompiler:
    def __init__(
        self,
        design: Union[
            sbol2.ComponentDefinition,
            sbol2.CombinatorialDerivation,
            sbol2.ModuleDefinition,
        ],
        sbh_registry: str,
        auth_token: str,
    ):
        self.design = design
        self.sbh = sbol2.PartShop(sbh_registry)
        self.sbh.key = auth_token

        self.plasmids = []
        self.backbones = []

    def index_collections(self, collections: List[str]):
        for uri in collections:
            temp_doc = sbol2.Document()
            self.sbh.pull(uri, temp_doc)

            for implementation in temp_doc.implementations:
                built_object = get_or_pull(temp_doc, self.sbh, implementation.built)
                if (
                    type(built_object) is sbol2.ModuleDefinition
                ):  # add type check for strain
                    self._extract_plasmids_from_strain(built_object, temp_doc)
                elif (
                    type(built_object) is sbol2.ComponentDefinition
                    and "http://identifiers.org/so/SO:0000637" in built_object.roles
                    and len(built_object.components) > 1
                ):  # add type check for plasmid or engineered plasmid
                    self.plasmids.append(Plasmid(built_object, None, temp_doc))

            for strain in temp_doc.moduleDefinitions:  # add type check for strain
                self._extract_plasmids_from_strain(strain, temp_doc)

            for component in (
                temp_doc.componentDefinitions
            ):  # add type check for plasmid or engineered plasmid
                if (
                    "http://identifiers.org/so/SO:0000637" in component.roles
                    and len(component.components) > 1
                ):
                    self.plasmids.append(Plasmid(component, None, temp_doc))

    def _extract_plasmids_from_strain(
        self, strain: sbol2.ModuleDefinition, doc: sbol2.Document
    ):
        for plasmid in strain.functionalComponents:
            plasmid_definition = get_or_pull(doc, self.sbh, plasmid.definition)
            self.plasmids.append(Plasmid(plasmid_definition, strain, doc))

In [13]:
auth = "88a75b47-8a9a-4084-b3e6-4f594b062636"
buildcompiler = BuildCompiler(None, "https://synbiohub.org", auth)

buildcompiler.index_collections(
    [
        "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1"
    ]
)

In [14]:
print(buildcompiler.plasmids)

[Plasmid:
  Name: pE0030_CD_C_D
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0030_CD/1
  Strain: None
  Fusion Sites: ['C', 'D']  Antibiotic Resistance: None
, Plasmid:
  Name: pJ23116_GB_B_G
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23116_GB/1
  Strain: None
  Fusion Sites: ['B', 'G']  Antibiotic Resistance: None
, Plasmid:
  Name: pJ23106_AB_A_B
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23106_AB/1
  Strain: None
  Fusion Sites: ['A', 'B']  Antibiotic Resistance: None
, Plasmid:
  Name: pE0040_CD_C_D
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0040_CD/1
  Strain: None
  Fusion Sites: ['C', 'D']  Antibiotic Resistance: None
, Plasmid:
  Name: pB0015_DH_D_H
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0015_DH/1
  Strain: None
  Fusion Sites: ['D', 'H']  Antibiotic Resistance: None
, Plasmid:
  Name: pJ23100_FB_B_F
  Definition: https://synbiohub.org/user/Gon/CID

# Create Collection of Implementations To Test

In [15]:
implementation_collection = sbol2.Document()
transformation_doc = sbol2.Document()
plas_doc = sbol2.Document()

buildcompiler.sbh.pull(
    "https://synbiohub.org/user/Gon/buildplanner_bacteerial_transformation/buildplanner_bacteerial_transformation_collection/1",
    transformation_doc,
)

buildcompiler.sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1",
    plas_doc,
)

In [16]:
implementation_collection.default_namespace = (
    "http://buildcompiler.org/implementations/"
)

dummy_activity = sbol2.Activity("plasmid_dna_extraction")
dummy_activity.name = "DNA extraction"
dummy_activity.types = "http://sbols.org/v2#build"

for plasmid in plas_doc.componentDefinitions:
    if "http://identifiers.org/so/SO:0000637" in plasmid.roles:
        implementation = sbol2.Implementation(f"{plasmid.displayId}_impl")
        implementation.built = plasmid.identity
        implementation.wasGeneratedBy = dummy_activity

        implementation_collection.add(implementation)

print(implementation_collection)

Design........................0
Build.........................0
Test..........................0
Analysis......................0
ComponentDefinition...........0
ModuleDefinition..............0
Model.........................0
Sequence......................0
Collection....................0
Activity......................0
Plan..........................0
Agent.........................0
Attachment....................0
CombinatorialDerivation.......0
Implementation................26
SampleRoster..................0
Experiment....................0
ExperimentalData..............0
Annotation Objects............0
---
Total: .........................26



In [17]:
implementation_collection.write("test.xml")

'Valid.'

In [18]:
impl_compiler = BuildCompiler(None, "https://synbiohub.org", auth)
impl_compiler.index_collections(
    [
        "https://synbiohub.org/user/Gon/impl_test/impl_test_collection/1",
        "https://synbiohub.org/user/Gon/buildplanner_bacteerial_transformation/buildplanner_bacteerial_transformation_collection/1",
    ]
)

print(impl_compiler.plasmids)

[Plasmid:
  Name: DVK_EF_E_F
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_EF/1
  Strain: None
  Fusion Sites: ['E', 'F']  Antibiotic Resistance: None
, Plasmid:
  Name: DVK_AE_A_E
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_AE/1
  Strain: None
  Fusion Sites: ['A', 'E']  Antibiotic Resistance: None
, Plasmid:
  Name: pE0040_CD_C_D
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0040_CD/1
  Strain: None
  Fusion Sites: ['C', 'D']  Antibiotic Resistance: None
, Plasmid:
  Name: pB0034_BC_B_C
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0034_BC/1
  Strain: None
  Fusion Sites: ['B', 'C']  Antibiotic Resistance: None
, Plasmid:
  Name: pJ23100_EB_B_E
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_EB/1
  Strain: None
  Fusion Sites: ['B', 'E']  Antibiotic Resistance: None
, Plasmid:
  Name: pJ23100_AB_A_B
  Definition: https://synbiohub.org/user/Gon/CIDARMoCloPlasmid